# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [13]:
 # Data Science
import pandas as pd
import numpy as np

# API Requests
from pprint import pprint
import requests
import json

# Data Viz
import hvplot.pandas
import matplotlib.pyplot as plt
import seaborn as sns
import folium

# Import API key
from api_keys_prof import geoapify_key

In [14]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Pressure,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Edinburgh of the Seven Seas,-37.0676,-12.3116,63.50,1009,92,96,27.13,SH,1735610943
1,1,West Island,-12.1568,96.8225,84.18,1011,79,20,5.75,CC,1735610943
2,2,Hadibu,12.6500,54.0333,71.94,1016,77,74,6.89,YE,1735610943
3,3,Arenápolis,-14.4503,-56.8461,71.71,1011,94,100,8.10,BR,1735610880
4,4,Grytviken,-54.2811,-36.5092,43.00,1013,78,93,8.14,GS,1735610943


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
m=folium.Map(location=[20,0],zoom_start=2)
for i,row in city_data_df.iterrows():
    folium.CircleMarker(
        location=(row["Lat"],row["Lng"]),
        radius=row["Humidity"]/20,
        popup=f"{row['City']}: {row['Humidity']}",
        color="blue", 
        fill=True,
        fill_color="blue"
    ).add_to(m)

m

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
idl_city_df=city_data_df[(city_data_df["Max Temp"]<35) & (city_data_df["Max Temp"]>10) & (city_data_df["Wind Speed"]<6) & (city_data_df["Cloudiness"]==0)]


# Drop any rows with null values
idl_city_df= idl_city_df.dropna()

# Display sample data
idl_city_df

,City_ID,City,Lat,Lng,Max Temp,Pressure,Humidity,Cloudiness,Wind Speed,Country,Date
109,109,Chamdo,31.1667,97.2333,19.89,1023,21,0,2.04,CN,1735610958
218,218,Nuuk,64.1835,-51.7216,16.88,1029,61,0,4.70,GL,1735610974
239,239,Petropavlovsk-Kamchatskiy,53.0452,158.6483,21.79,994,42,0,2.24,RU,1735610977
344,344,Tuluceşti,45.5667,28.0333,33.51,1030,88,0,4.90,RO,1735610992
412,412,Undurkhaan,47.3194,110.6556,10.04,1024,57,0,4.85,MN,1735611003
468,468,Khorramabad,33.4878,48.3558,29.82,1020,93,0,2.30,IR,1735611012
493,493,Zhangjiakou,40.8100,114.8794,18.48,1028,55,0,1.28,CN,1735611016


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = idl_city_df.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
109,Chamdo,CN,31.1667,97.2333,21,
218,Nuuk,GL,64.1835,-51.7216,61,
239,Petropavlovsk-Kamchatskiy,RU,53.0452,158.6483,42,
344,Tuluceşti,RO,45.5667,28.0333,88,
412,Undurkhaan,MN,47.3194,110.6556,57,
468,Khorramabad,IR,33.4878,48.3558,93,
493,Zhangjiakou,CN,40.8100,114.8794,55,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "limit": 1, 
    "categories": "accommodation.hotel", 
    "sort": "distance",  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    city = row["City"]
    country = row["Country"]
    latitude = row['Lat']
    longitude = row['Lng']
    

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary

    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    data = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = data["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Chamdo - nearest hotel: 银杏商务宾馆
Nuuk - nearest hotel: Nordbo
Petropavlovsk-Kamchatskiy - nearest hotel: ООО Постоялый двор
Tuluceşti - nearest hotel: No hotel found
Undurkhaan - nearest hotel: Khanburged Hotel
Khorramabad - nearest hotel: هتل رنگین کمان
Zhangjiakou - nearest hotel: 商务宾馆


,City,Country,Lat,Lng,Humidity,Hotel Name
109,Chamdo,CN,31.1667,97.2333,21,银杏商务宾馆
218,Nuuk,GL,64.1835,-51.7216,61,Nordbo
239,Petropavlovsk-Kamchatskiy,RU,53.0452,158.6483,42,ООО Постоялый двор
344,Tuluceşti,RO,45.5667,28.0333,88,No hotel found
412,Undurkhaan,MN,47.3194,110.6556,57,Khanburged Hotel
468,Khorramabad,IR,33.4878,48.3558,93,هتل رنگین کمان
493,Zhangjiakou,CN,40.8100,114.8794,55,商务宾馆


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:


m=folium.Map(location=[20,0],zoom_start=2)
for i,row in hotel_df.iterrows():
    folium.CircleMarker(
        location=(row["Lat"],row["Lng"]),
        radius=row["Humidity"]/20,
        popup=f"{row['City']}: {row['Humidity']}\nHotel: {row['Hotel Name']}",
        color="blue", 
        fill=True,
        fill_color="blue"
    ).add_to(m)

m